In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
import matplotlib.pyplot as plt
import folium
from geopy.geocoders import Nominatim
import time
import numpy as np

In [3]:
def scrape_water_data(url, table_div_id="tablecontainer"):
    """Scrape water monitoring data table with embedded links."""
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")
    container = soup.find("div", id=table_div_id)
    if not container:
        raise ValueError(f"Container '{table_div_id}' not found")

    table = container.find("table")
    if not table:
        raise ValueError("No table found in container")

    # Extract column headers
    header_rows = table.find("thead").find_all("tr")
    columns = [th.get_text(strip=True) for th in header_rows[-1].find_all("th")]

    # Extract table data with links
    data = []
    for row in table.find("tbody").find_all("tr"):
        cells = row.find_all("td")
        if not cells:
            continue

        row_data = [cell.get_text(strip=True) for cell in cells]

        # Extract site link (first column)
        site_url = None
        if cells and (link := cells[0].find("a")) and "href" in link.attrs:
            site_url = urljoin(url, link["href"])

        # Extract station link (third column)
        station_url = None
        if len(cells) > 2 and (link := cells[2].find("a")) and "href" in link.attrs:
            station_url = urljoin(url, link["href"])

        row_data.extend([site_url, station_url])
        data.append(row_data)

    columns.extend(["SITE_URL", "STATION_URL"])
    return pd.DataFrame(data, columns=columns)

In [4]:
base_url = "https://kumina.water.wa.gov.au/waterinformation/telem"

river_data = scrape_water_data(f"{base_url}/stage.cfm")


rain_data = scrape_water_data(f"{base_url}/rain.cfm")

In [5]:
river_data

,SITE,RIVER,STATION NAME,TIME/DATE,STAGE,RIVER LEVEL,HRLY RATE,TENDENCY,MAX REC LEVEL,DATE,SITE_URL,STATION_URL
0,601001601001 - Neds CornerComments:No additona...,Young River,Neds Corner,07:05 27Aug2025,10.536,0.511,0.005,STEADY,15.193,05JAN2007,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
1,601004601004 - FairfieldComments:No additonal ...,Lort River,Fairfield,07:05 27Aug2025,10.437,0.253,0.000,STEADY,15.116,09FEB2017,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
2,601005601005 - CascadesComments:No additonal c...,Young River,Cascades,07:00 27Aug2025,10.059,0.067,0.000,STEADY,12.901,19FEB2018,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
3,601008601008 - Myrup RdComments:Road level = 1...,Coramup Creek,Myrup Rd,07:00 27Aug2025,10.188,0.133,0.003,STEADY,13.839,06JAN1999,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
4,601009601009 - Fisheries RdComments:Road level...,Bandy Creek,Fisheries Rd,07:00 27Aug2025,10.216,0.185,0.000,STEADY,13.425,05JAN2007,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
...,...,...,...,...,...,...,...,...,...,...,...,...
241,809322809322 - Odonnell RangeComments:No addit...,Wilson River,Odonnell Range,07:05 27Aug2025,9.704,-0.785,0.000,STEADY,23.447,14MAR2011,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
242,809339809339 - Tarrara BarComments:Peak: 22Feb...,Ord River,Tarrara Bar,08:00 27Aug2025,11.119,1.119,0.000,STEADY,23.490,25MAR2006,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
243,809340809340 - Flying Fox HoleComments:2002 Pe...,Dunham River,Flying Fox Hole,07:00 27Aug2025,10.535,0.057,0.001,STEADY,22.459,22FEB2002,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
244,810001810001 - The HillComments:No additonal c...,Gumm Creek,The Hill,06:05 27Aug2025,9.825,-0.064,0.000,STEADY,14.933,04MAR2000,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...


In [5]:
rain_data

,SITE,RIVER,STATION NAME,TIME/DATE,SinceLast 09:00,24 Hr toLast 09:00,24 Hr toPrev 09:00,3 DAYTOTAL,MAX RECRAINFALL,DATE,SITE_URL,STATION_URL
0,501008501008 - No 1 MoochalabraComments:No add...,Moochalabra Creek,No 1 Moochalabra,06:00 26Aug2025,0.0,0.0,0.0,0.0,254.9,21DEC2008,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
1,501029501029 - Moochalabra DamComments:GS 809 318,Moochalabra Creek,Moochalabra Dam,09:00 26Aug2025,0.0,0.0,0.0,0.0,197.2,21DEC2008,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
2,502001502001 - Mt WinifredComments:GS 802 202,Leopold River,Mt Winifred,09:05 26Aug2025,0.0,0.0,0.0,0.0,165.8,24DEC1987,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
3,502002502002 - Mud SpringsComments:No additona...,Leopold River,Mud Springs,06:00 26Aug2025,0.0,0.0,0.0,0.0,189.9,21FEB1991,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
4,502005502005 - Mt KraussComments:GS 802 203,Margaret River,Mt Krauss,09:05 26Aug2025,0.0,0.0,0.0,0.0,195.6,28JAN2018,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
...,...,...,...,...,...,...,...,...,...,...,...,...
156,510252510252 - Kwolyn HillComments:GS 615 012,Lockhart River,Kwolyn Hill,09:05 26Aug2025,0.0,3.4,3.2,6.6,87.4,22JAN2000,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
157,510254510254 - Lake Toolibin InflowComments:GS...,Northern Arthur River,Lake Toolibin Inflow,09:00 26Aug2025,0.0,0.2,3.8,4.0,110.8,29JAN1990,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
158,510598510598 - Waterhatch BridgeComments:No ad...,Dale River,Waterhatch Bridge,09:00 26Aug2025,0.0,3.2,29.0,32.2,79.8,10FEB2017,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
159,512233512233 - CascadesComments:No additonal c...,Young River,Cascades,09:00 26Aug2025,0.0,0.0,2.0,2.0,92.6,14APR2025,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...


In [6]:
rain_data["STATION NAME"].unique()

array(['No 1 Moochalabra', 'Moochalabra Dam', 'Mt Winifred',
       'Mud Springs', 'Mt Krauss', 'Me No Savvy', 'The Hill',
       'Frog Hollow', 'Bedford Downs', 'Liamma', 'Elgee Cliffs',
       'Phillips Range', 'Durack Range', 'Old Ord Homestead',
       'Dunham Gorge', 'Lake Kununurra', 'Homestead', 'Mt.Rob.',
       'Eight Mile Mill', 'Microwave Tower', 'Margaret Gorge',
       'Abney Hill', 'Dimond Gorge', 'Mt Joseph', 'Dales Yard',
       'Ellendale', 'Noonkanbah', 'Coonanarrina Pool', 'North Pole U/S',
       'Soansville', 'Abydos North', 'Pincunah', 'Carraba', 'Marble Bar',
       'Ripon Hills Road', 'Nullagine', 'Tumbinna Pool',
       'Lyre Creek Well', 'Black Hills', 'Munjina', 'Upper Portland',
       'Gregorys Gorge', 'Flat Rocks', '56A', '13A', 'Cadgerina Pool',
       'Poonda', 'Tarina', 'Waterloo', 'Rundalls', 'Paradise Creek',
       'Bilanoo Pool', 'Air Strip', 'Meedo Pool', 'Fishy Pool', 'Minilya',
       'Jimba', 'Yinnethara Crossing', 'Capricorn Range', 'Newman',
 

In [ ]:
def quick_geocode(station_names):
    geolocator = Nominatim(user_agent="wa_rainfall")
    results = []

    for station in station_names:
        if pd.isna(station):
            continue
        try:
            loc = geolocator.geocode(f"{station}, Western Australia")
            if loc:
                results.append(
                    {"station": station, "lat": loc.latitude, "lon": loc.longitude}
                )
            time.sleep(1)
        except:
            pass

    return pd.DataFrame(results)


coords = quick_geocode(rain_data["STATION NAME"].unique())

In [11]:
coords

,station,lat,lon
0,Moochalabra Dam,-15.622069,128.101063
1,Mt Winifred,-18.054790,126.281870
2,Mud Springs,-15.879386,128.771506
3,Mt Krauss,-18.321220,126.121690
4,The Hill,-31.951656,115.847289
...,...,...,...
76,Jelcobine,-32.406830,116.766200
77,Frenches,-31.564623,116.656291
78,Piesseville,-33.194187,117.286424
79,Waterhatch Bridge,-32.116662,116.835555


In [12]:
# Create map centered on Western Australia
wa_map = folium.Map(location=[-26.0, 121.0], zoom_start=5, tiles="OpenStreetMap")

# Remove rows with missing coordinates
coords_clean = coords.dropna(subset=["lat", "lon"])

# Add markers for each station
for idx, row in coords_clean.iterrows():
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=6,
        popup=f"<b>{row['station']}</b><br>Lat: {row['lat']:.4f}<br>Lon: {row['lon']:.4f}",
        tooltip=row["station"],
        color="white",
        weight=2,
        fillColor="red",
        fillOpacity=0.7,
    ).add_to(wa_map)

# Save the map
wa_map.save("wa_rainfall_stations.html")
print(f"Plotted {len(coords_clean)} rainfall stations")

# Display the map
wa_map

Plotted 81 rainfall stations


In [6]:
river_data

,SITE,RIVER,STATION NAME,TIME/DATE,STAGE,RIVER LEVEL,HRLY RATE,TENDENCY,MAX REC LEVEL,DATE,SITE_URL,STATION_URL
0,601001601001 - Neds CornerComments:No additona...,Young River,Neds Corner,07:05 27Aug2025,10.536,0.511,0.005,STEADY,15.193,05JAN2007,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
1,601004601004 - FairfieldComments:No additonal ...,Lort River,Fairfield,07:05 27Aug2025,10.437,0.253,0.000,STEADY,15.116,09FEB2017,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
2,601005601005 - CascadesComments:No additonal c...,Young River,Cascades,07:00 27Aug2025,10.059,0.067,0.000,STEADY,12.901,19FEB2018,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
3,601008601008 - Myrup RdComments:Road level = 1...,Coramup Creek,Myrup Rd,07:00 27Aug2025,10.188,0.133,0.003,STEADY,13.839,06JAN1999,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
4,601009601009 - Fisheries RdComments:Road level...,Bandy Creek,Fisheries Rd,07:00 27Aug2025,10.216,0.185,0.000,STEADY,13.425,05JAN2007,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
...,...,...,...,...,...,...,...,...,...,...,...,...
241,809322809322 - Odonnell RangeComments:No addit...,Wilson River,Odonnell Range,07:05 27Aug2025,9.704,-0.785,0.000,STEADY,23.447,14MAR2011,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
242,809339809339 - Tarrara BarComments:Peak: 22Feb...,Ord River,Tarrara Bar,08:00 27Aug2025,11.119,1.119,0.000,STEADY,23.490,25MAR2006,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
243,809340809340 - Flying Fox HoleComments:2002 Pe...,Dunham River,Flying Fox Hole,07:00 27Aug2025,10.535,0.057,0.001,STEADY,22.459,22FEB2002,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
244,810001810001 - The HillComments:No additonal c...,Gumm Creek,The Hill,06:05 27Aug2025,9.825,-0.064,0.000,STEADY,14.933,04MAR2000,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...


In [20]:
for col in ["STAGE", "RIVER LEVEL", "HRLY RATE", "MAX REC LEVEL"]:
    river_data[col] = pd.to_numeric(river_data[col], errors="coerce")

In [44]:
merged_df = pd.merge(river_data, rain_data, on="RIVER", how="inner")
merged_df

,SITE_x,RIVER,STATION NAME_x,TIME/DATE_x,STAGE,RIVER LEVEL,HRLY RATE,TENDENCY,MAX REC LEVEL,DATE_x,...,STATION NAME_y,TIME/DATE_y,SinceLast 09:00,24 Hr toLast 09:00,24 Hr toPrev 09:00,3 DAYTOTAL,MAX RECRAINFALL,DATE_y,SITE_URL_y,STATION_URL_y
0,601001601001 - Neds CornerComments:No additona...,Young River,Neds Corner,07:05 27Aug2025,10.536,0.511,0.005,STEADY,15.193,05JAN2007,...,Neds Corner,07:05 27Aug2025,4.6,0.0,1.0,5.6,134.0,05JAN2007,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
1,601001601001 - Neds CornerComments:No additona...,Young River,Neds Corner,07:05 27Aug2025,10.536,0.511,0.005,STEADY,15.193,05JAN2007,...,Cascades,07:00 27Aug2025,2.2,0.0,2.0,4.2,92.6,14APR2025,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
2,601004601004 - FairfieldComments:No additonal ...,Lort River,Fairfield,07:05 27Aug2025,10.437,0.253,0.000,STEADY,15.116,09FEB2017,...,Fairfield,07:05 27Aug2025,6.8,0.2,1.2,8.2,118.4,14APR2025,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
3,601005601005 - CascadesComments:No additonal c...,Young River,Cascades,07:00 27Aug2025,10.059,0.067,0.000,STEADY,12.901,19FEB2018,...,Neds Corner,07:05 27Aug2025,4.6,0.0,1.0,5.6,134.0,05JAN2007,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
4,601005601005 - CascadesComments:No additonal c...,Young River,Cascades,07:00 27Aug2025,10.059,0.067,0.000,STEADY,12.901,19FEB2018,...,Cascades,07:00 27Aug2025,2.2,0.0,2.0,4.2,92.6,14APR2025,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,809339809339 - Tarrara BarComments:Peak: 22Feb...,Ord River,Tarrara Bar,08:00 27Aug2025,11.119,1.119,0.000,STEADY,23.490,25MAR2006,...,Lake Kununurra,06:00 27Aug2025,0.0,0.0,0.0,0.0,223.9,08FEB2014,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
205,809340809340 - Flying Fox HoleComments:2002 Pe...,Dunham River,Flying Fox Hole,07:00 27Aug2025,10.535,0.057,0.001,STEADY,22.459,22FEB2002,...,Liamma,06:10 27Aug2025,0.0,0.0,0.0,0.0,169.0,13MAR2011,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
206,809340809340 - Flying Fox HoleComments:2002 Pe...,Dunham River,Flying Fox Hole,07:00 27Aug2025,10.535,0.057,0.001,STEADY,22.459,22FEB2002,...,Dunham Gorge,07:05 27Aug2025,0.0,0.0,0.0,0.0,180.6,17MAR2005,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
207,810001810001 - The HillComments:No additonal c...,Gumm Creek,The Hill,06:05 27Aug2025,9.825,-0.064,0.000,STEADY,14.933,04MAR2000,...,The Hill,06:05 27Aug2025,0.0,0.0,0.0,0.0,191.8,20DEC2020,https://kumina.water.wa.gov.au/waterinformatio...,https://kumina.water.wa.gov.au/waterinformatio...
